## Importando Dependências

In [ ]:
try:
    # Importing the OpenAI library
    import openai
    import re
    import json

except ModuleNotFoundError:
    # Downloading the OpenAI library if it has not been downloaded yet
    !pip install -q openai
    import openai
    import re
    import json

## Entering the OpenAI API key

In [ ]:
openai.api_key = "PASTE-HERE-YOUR-KEY"
MODELO = "gpt-3.5-turbo" # Options: "text-davinci-003", "gpt-3.5-turbo"

## Defining Basic Information for the Algorithm

In [ ]:
# Defines difficulty levels until the code reaches the end of the evaluation
# This is kept in Portuguese so as not to change the original code
niveis_dificuldade = ["1 - Fácil", "2 - Médio", "3 - Difícil", 'Fim']


# Instructs the algorithm to always start with an easy level question
dificuldade = '1 - Fácil'


# Stores the student's trajectory of mistakes and successes
trajetoria = []

## Receiving Context Information

In [ ]:
contexto = input("Enter the class topic text: ")


prompt_base = f'''
I want you to act as an educational assistant for teachers.

Your objective is to read a base text for a class informed by the teacher and create
discursive questions for students according to the following difficulty levels:
1 - Fácil
2 - Médio
3 - Difícil

When requested, you must also inform the level of correctness of the answers
informed according to the numbers corresponding to the following options:
1 - Resposta Errada
2 - Resposta Parcialmente Correta
3 - Resposta Correta

The basic text of the class is as follows:

"
{contexto}
"
'''

## Asking Student Adaptive Questions

---



In [ ]:
while dificuldade != "Fim":
    # Assemble the question text for GPT
    pergunta_chatgpt = f'''
    {prompt_base}

    According to the context above, generate a question with difficulty level {dificuldade}.

    Your answer must contain only the wording of the question. The difficulty level or other information should be omitted.
    '''
    # Requests GPT to generate a question according to the selected base text and difficulty level
    prompt_chatgpt = openai.ChatCompletion.create(
        model=MODELO,
        messages=[
            {"role": "user", "content": pergunta_chatgpt},
        ])
    pergunta_gerada = "{}".format(prompt_chatgpt.choices[0]["message"]["content"])


    # Displays the generated question and collects the answer
    print("\n=============================================================")
    print("=============================================================")
    print("=============================================================")
    print(pergunta_gerada)
    resposta = input("Enter the answer to the question above: ")



    pergunta_chatgpt = f'''
    {prompt_base}

    The teacher asked the student the following question:
    "{pergunta_gerada}"

    The student reported the following response:
    "{resposta}"

    Inform the level of correctness of the student's answer.

    Your answer must necessarily inform only the
    number of the correctness option among the options provided above.
    '''
    # Asks the question about the correctness of the student's answer to the GPT
    prompt_chatgpt = openai.ChatCompletion.create(
        model=MODELO,
        messages=[
            {"role": "user", "content": pergunta_chatgpt},
        ])


    # Parses the correctness of the student's response
    corretude_resposta = "{}".format(prompt_chatgpt.choices[0]["message"]["content"])
    numero_corretude_resposta = int(re.findall('[0-9]+', corretude_resposta)[0])


    # Increases the student's level of difficulty if they get the question right
    if numero_corretude_resposta == 3:
        nova_dificuldade = niveis_dificuldade[niveis_dificuldade.index(dificuldade) + 1]
    else:
        nova_dificuldade = dificuldade


    # Requests GPT to analyze the mood of the student's response
    pergunta_humor_chatgpt = f'''
    {prompt_base}

    The teacher asked the student the following question:
    "{pergunta_gerada}"

    The student reported the following response:
    "{resposta}"

    Analyze the answer provided by the student and inform the mood of the
    answer according to the numbers corresponding to the following options:
    1. Humor Negativo
    2. Humor Neutro
    3. Humor Positivo
    '''
    prompt_humor_chatgpt = openai.ChatCompletion.create(
        model=MODELO,
        messages=[
            {"role": "user", "content": pergunta_humor_chatgpt},
        ])
    # Parses the mood of the student's response
    humor_resposta = "{}".format(prompt_humor_chatgpt.choices[0]["message"]["content"])

    # Requests that GPT analyzes the sentiment of the student's responseo
    pergunta_humor_chatgpt = f'''
    {prompt_base}

    The teacher asked the student the following question:
    "{pergunta_gerada}"

    The student reported the following response:
    "{resposta}"

    Analyze the answer provided by the student and inform the feeling expressed by the answer.
    Your response in relation to the student's feeling in the response must be represented in a single
    word (por exemplo: "Confiança", "Insegurança", "Empenho", "Preguiça", "Felicidade", "Alegria",
    "Entusiasmo", "Gratidão", "Esperança", "Satisfação", "Amor", "Tristeza", "Raiva", "Frustração",
    "Medo", "Decepção", "Culpa", "Ansiedade", etc.).
    '''
    prompt_sentimento_chatgpt = openai.ChatCompletion.create(
        model=MODELO,
        messages=[
            {"role": "user", "content": pergunta_humor_chatgpt},
        ])
    # Parses the mood of the student's response
    sentimento_resposta = "{}".format(prompt_sentimento_chatgpt.choices[0]["message"]["content"])


    # Stores student progress (trajectory)
    trajetoria.append(
        {
             "dificuldade": dificuldade,
             "corretude": corretude_resposta,
             "humor": humor_resposta,
             "sentimento": sentimento_resposta,
        }
    )

    # Updates the student's difficulty level
    dificuldade = nova_dificuldade


## Show trajectory

----

In [ ]:
print("=============================")
print("==== STUDENT TRAJECTORY =+===")
print("=============================")

print(f"Number of Questions: {len(trajetoria)}\n")

for numero_pergunta, resposta in enumerate(trajetoria, 1):
    dificuldade = resposta["dificuldade"]
    corretude = resposta["corretude"]
    humor = resposta["humor"]
    sentimento = resposta["sentimento"]

    print(f"=== Question {numero_pergunta}")
    print(f"\Difficulty: {dificuldade}")
    print(f"\tCorrectness: {corretude}")
    print(f"\tMood: {humor}")
    print(f"\tFeelings: {sentimento}")
    
    print("\n")